In [3]:
import urllib
from bs4 import BeautifulSoup

url="http://web.mta.info/developers/turnstile.html"  #url
u=urllib.request.urlopen(url)
html=u.read()

soup=BeautifulSoup(html,"html.parser")
links= soup.find_all('a')

count=0

for link in links:
    
    if '1706' in str(link.get('href')):
        print("http://web.mta.info/developers/"+
              link.get('href'),"turnstile_100617"+str(count)+'.txt')
        count=count+1

http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt turnstile_1006170.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170617.txt turnstile_1006171.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170610.txt turnstile_1006172.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_170603.txt turnstile_1006173.txt


In [5]:
import pandas as pd
filename="turnstile_data_master_with_weather(1).csv"
weather_data=pd.read_csv(filename)
weather_data.head()

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,0,R001,01-05-2011,01:00:00,1,REGULAR,0,0,30.31,42,...,39,30.27,0,0,5,50,60,69,0.0,0
1,1,R001,01-05-2011,05:00:00,5,REGULAR,217,553,30.31,42,...,39,30.27,0,0,5,50,60,69,0.0,0
2,2,R001,01-05-2011,09:00:00,9,REGULAR,890,1262,30.31,42,...,39,30.27,0,0,5,50,60,69,0.0,0
3,3,R001,01-05-2011,13:00:00,13,REGULAR,2451,3708,30.31,42,...,39,30.27,0,0,5,50,60,69,0.0,0
4,4,R001,01-05-2011,17:00:00,17,REGULAR,4400,2501,30.31,42,...,39,30.27,0,0,5,50,60,69,0.0,0


In [5]:
import pandas
import pandasql

def num_rainy_days(filename):
    weather_data=pandas.read_csv(filename)
    
    q="""
    SELECT COUNT(*)
    FROM weather_data
    WHERE rain=1;
    """
    rainy_days=pandasql.sqldf(q.lower(),locals()) #pandasql converts csv file to sql database
    #locals= local scope (i.e only in this function)
    return rainy_days
csv_data="turnstile_data_master_with_weather(1).csv"
print(num_rainy_days(csv_data))

   count(*)
0     44104


In [1]:
import sys

def mapper(): #combines data to form cluster
    for line in sys.stdin:
        data=line.strip().split(",")
        if len(data)==22:
            Unnamed,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder=data
            print("{0}\t{1}".format(UNIT, ENTRIESn_hourly))
            
sys.stdin=open('turnstile_data_master_with_weather(1).csv')
sys.stdout=open('mapper_result.text','w')
mapper()

In [4]:

import sys
def reducer():
    entriesTotal=0
    oldKey=None
    for line in sys.stdin:
        data=line.strip().split("\t") #break the line where there is tab
        if len(data)!=2:
            continue
        thisKey,thisEntry=data
        if oldKey and oldKey!=thisKey: #if old key exists and oldkey!=thiskey, create new key value pair
            print("{0}\t{1}".format(oldKey,entriesTotal))
            entriesTotal=0
            
        oldKey=thisKey
        try:
            entriesTotal+=float(thisEntry)
        except:
            print(thisEntry)
    if oldKey is None:        
        print("{0}\t{i}".format(oldKey,entriesTotal))
        
sys.stdin=open('mapper_result.text','r')
sys.stdout=open('reducer_result.txt','w')
reducer()